In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
import osmnx
import openrouteservice
import requests

In [ ]:
porto_area = osmnx.geocode_to_gdf('Porto, Portugal')

porto_area.plot()

center_porto = porto_area['geometry'].centroid

import folium
from folium.map import *
from openrouteservice import client

# openrouteservice api key
api_key = "5b3ce3597851110001cf6248516d2c7fa9b240a1989c451ecb3b2520"  
clnt = client.Client(key=api_key)

m = folium.Map(location=[41.16181,-8.62057], zoom_start=14)

m

In [ ]:
tmp = requests.get('https://wdl-data.fra1.digitaloceanspaces.com/porto-digital/POIs.json').json()
ret = []

for count, point_of_interest in enumerate(tmp['points_of_interest']):
    category = point_of_interest.get('category')[0]['value']
    location = point_of_interest.get('location')
    created = point_of_interest.get('created')
    
    if 'point' in location:
        location = location.get('point')[0].get('Point').get('posList')
    else:
        location = np.nan
    description = point_of_interest.get('description')[0]['value']
    
    ret.append({
        'id': count,
        'created_at': created,
        'category': category,
        'location': location,
        'description': description
    })

df_pois = pd.DataFrame.from_records(ret)
df_pois

In [ ]:
split = df_pois['location'].str.split(' ', 1, expand=True)
df_split = df_pois.assign(latitude=split[0], longitude=split[1])
df_split.drop('location', 1, inplace=True)

In [ ]:
#import sys
#sys.setrecursionlimit(10**6)

for indice, row in df_split.iterrows():
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        popup=row['category']
        ).add_to(m)

m